In [1]:
import numpy as np
import csv
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('movie_metadata.csv')

to_drop = ['director_name', 'num_critic_for_reviews', 'actor_2_name','actor_1_name', 'movie_title', 'num_voted_users'
           , 'actor_3_name', 'plot_keywords', 'movie_imdb_link', 'num_user_for_reviews']
          
features_list = data.columns.difference(to_drop)
movie_data = data[features_list]
#print(np.sum(movie_num.isnull()))
#print(movie_data.content_rating.unique())
pd.options.mode.chained_assignment = None 

        
print('Done')

Done


# Deleting NaN's

In [2]:
NA_THRESH = 4

def remove_too_many_NaN(data, threshold):
    print("Length before deletion: {}".format(len(data)))

    remove_indices = []
    for index, nNaN in data.isnull().sum(axis=1).iteritems():
        if nNaN >= threshold:
            remove_indices.append(index)
    
    # drop movies with too many NaNs
    data = data.drop(data.index[remove_indices])
    print("Length after deletion: {}".format(len(data)))
    
    return data


movie_data = remove_too_many_NaN(movie_data, NA_THRESH)

Length before deletion: 5043
Length after deletion: 4900


# Changing Color to Numeric

In [3]:
new_color_column = movie_data.color.replace(to_replace = 'Color', value = 1)

for item in new_color_column:
    if item != 1:
        new_color_column = new_color_column.replace(to_replace = item, value = 0)
        
movie_data.color = new_color_column 
#print(movie_data['color'])

pd.options.mode.chained_assignment= None
print('Done')

Done


# Changing Country to Numeric

In [4]:
movie_data.country = movie_data.country.replace(to_replace ='USA', value = 1)

for item in movie_data["country"]:
    if item != 1:
        movie_data.country = movie_data.country.replace(to_replace = item, value = 0)
#print(movie_data['country'])

pd.options.mode.chained_assignment= None
print('Done')

Done


# Changing Languange to Numeric

In [5]:
new_language_column = movie_data.language.replace(to_replace = 'English', value = 1)

for item in new_language_column:
    if item != 1:
        new_language_column = new_language_column.replace(to_replace = item, value = 0)
        
movie_data.language = new_language_column 
print(movie_data['language'])

pd.options.mode.chained_assignment= None
print('Done')

0       1
1       1
2       1
3       1
5       1
6       1
7       1
8       1
9       1
10      1
11      1
12      1
13      1
14      1
15      1
16      1
17      1
18      1
19      1
20      1
21      1
22      1
23      1
24      1
25      1
26      1
27      1
28      1
29      1
30      1
       ..
5008    1
5009    1
5010    1
5011    1
5012    1
5013    1
5014    1
5015    1
5016    1
5017    1
5018    1
5019    1
5021    1
5022    0
5023    1
5024    1
5025    1
5026    0
5027    0
5028    1
5029    0
5031    1
5033    1
5034    1
5035    0
5036    1
5037    1
5040    1
5041    1
5042    1
Name: language, dtype: int64
Done


# Changing Genres into individual Columns

In [6]:
genres = []

for string in movie_data['genres']:
    genre = string.split('|')
    genres = genres+genre
    
print(set(genres))

set(['Sci-Fi', 'Crime', 'Romance', 'Animation', 'Music', 'Comedy', 'War', 'Horror', 'Film-Noir', 'Adventure', 'News', 'Thriller', 'Western', 'Mystery', 'Short', 'Drama', 'Action', 'Documentary', 'Musical', 'History', 'Family', 'Fantasy', 'Sport', 'Biography'])


# Chaning Content_Rating to Numeric

In [ ]:
def content_to_numerical(data):
    data = data.replace(to_replace='G', value=0)
    data = data.replace(to_replace='PG', value=12)
    data = data.replace(to_replace='PG-13', value=13)
    data = data.replace(to_replace='R', value=17)
    data = data.replace(to_replace='NC-17', value=17)
    
    data = data.replace(to_replace='TV-PG', value=12)
    data = data.replace(to_replace='TV-MA', value=17)
    data = data.replace(to_replace='TV-G', value=0)
    data = data.replace(to_replace='TV-Y', value=0)
    data = data.replace(to_replace='TV-Y7', value=7)
    data = data.replace(to_replace='TV-14', value=14)
    
    data = data.replace(to_replace='Not Rated', value=0)
    data = data.replace(to_replace='Unrated', value=0)
    data = data.replace(to_replace='Approved', value=0)
    data = data.replace(to_replace='Passed', value=0)
    
    data = data.replace(to_replace='X', value=17)
    data = data.replace(to_replace='M', value=17)
    data = data.replace(to_replace='GP', value=12)
    
    return data

movie_data = content_to_numerical(movie_data)